In [ ]:
import siibra
import numpy as np
from nilearn import plotting

In [ ]:
difumo64,longbundles,jubrain = [
    siibra.parcellations[n] 
    for n in ["difumo64","long bundles","julich"]]

funcmap,bundlemap,cytomap = [
    p.get_map("mni152","continuous") 
    for p in [difumo64,longbundles,jubrain]]

In [ ]:
funcindex = 1
funcregionmap = funcmap.fetch(mapindex=funcindex)
funcregion = funcmap.decode_label(mapindex=funcindex)
plotting.plot_stat_map(funcregionmap,title=f"Source: {funcregion.name}")

assignments = cytomap.assign(funcregionmap)
for region,pmap,scores in assignments:
    if scores['overlap']>=0.1:
        view = plotting.plot_stat_map(pmap,
            title=f"Target: {region.name} ({scores['overlap']*100:.1f}%)")
        view.add_contours(funcregionmap)

In [ ]:
overlaps = np.zeros((len(funcmap),len(bundlemap)))

for funcindex,funcregionmap in enumerate(funcmap.fetch_iter()):
    funcregion = funcmap.decode_label(funcindex)
    assignments = bundlemap.assign(funcregionmap,
                                   msg=f"{funcindex+1:2d}/{len(funcmap):2d}: {funcregion.name:30.30}")
    for bundleregion,pmap,scores in assignments:
        bundleindex = bundlemap.decode_region(bundleregion)[0].map
        overlaps[funcindex,bundleindex] = scores['overlap']

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

fig = plt.figure(figsize=(12, 16),)
ax  = fig.add_subplot(211)
ax.imshow(overlaps.T,interpolation='nearest',cmap='gray_r')

bundlenames = [bundlemap.decode_label(i).name for i in range(len(bundlemap))]
ax.set_yticks(range(len(bundlenames)))
ax.set_yticklabels(bundlenames)

areanames = [funcmap.decode_label(i).name for i in range(len(funcmap))]
ax.set_xticks(range(len(areanames)))
ax.set_xticklabels(areanames,rotation=90,fontsize=10)
ax.grid(True)
pass